# Scripts just to test YOLO models

## Train on random weights

In [ ]:
from ultralytics import YOLO
model = YOLO("yolo12n.pt")

results = model.train(data="coco8.yaml", epochs=5, imgsz=640, device='cuda')

## Train on custom dataset

### Yolo

In [ ]:
from ultralytics import YOLO
model = YOLO("yolo12n.pt")

results = model.train(data="/home/lucas-fontoura/Documents/yolo-drone/data/datasets/veryreduced-sliced-sahi-yolodrone/data.yaml", epochs=10, imgsz=640, device='cuda', batch=16)

In [ ]:
metrics = model.val()

In [ ]:
results = {
    "map50-95 (B)": metrics.box.map,
    "map50 (B)": metrics.box.map50,
    "map75 (B)": metrics.box.map75,
    "map50-95 (B) of each category": metrics.box.maps,
}

results

In [ ]:
# Predict with the model
results = model("/home/lucas-fontoura/Documents/yolo-drone/data/datasets/VeryReduced_InfectedLeaves/images/Aug03_USAMV_DJI_0562_f_01100.jpg", save=True)  # predict on an image

# Access the results
for result in results:
    xy = result.masks.xy  # mask in polygon format
    xyn = result.masks.xyn  # normalized
    masks = result.masks.data  # mask in matrix format (num_objects x H x W)

### Yolo-seg

In [ ]:
from ultralytics import YOLO
model = YOLO("yolo11n-seg.pt")

results = model.train(data="/home/lucas-fontoura/Documents/yolo-drone/data/datasets/synthetic-seg/data.yaml", epochs=50, imgsz=800, device='cuda', batch=16)

In [ ]:
metrics = model.val()

In [ ]:
results = {
    "map50-95 (B)": metrics.seg.map,
    "map50 (B)": metrics.seg.map50,
    "map75 (B)": metrics.seg.map75,
    "map50-95 (B) of each category": metrics.seg.maps,
}

results

In [ ]:
# Predict with the model
results = model("/home/lucas-fontoura/Documents/yolo-drone/data/datasets/VeryReduced_InfectedLeaves/images/Aug03_USAMV_DJI_0562_f_01100.jpg", save=True)  # predict on an image

# Access the results
for result in results:
    xy = result.masks.xy  # mask in polygon format
    xyn = result.masks.xyn  # normalized
    masks = result.masks.data  # mask in matrix format (num_objects x H x W)